In [1]:
import pandas as pd
from pandas import DataFrame as df
import seaborn as sns
import sys
import numpy as np
import time

%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.externals import joblib
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier

sns.set_style('ticks')
sys.path.append("/home/mschlupp/pythonTools")

In [2]:
ga_data = pd.read_csv("files/finalSets/ga_noEvts.csv"
                      , usecols=["device_id","group"
                                , "phone_brand","isTrain"
                                , "device_model"])

## Encode the  colums 

In [3]:
enc_brand = LabelEncoder()
enc_device = LabelEncoder()
enc_group = LabelEncoder()

In [4]:
ga_data["phone_brand"] = enc_brand.fit_transform(ga_data.phone_brand)
ga_data["device_model"] = enc_device.fit_transform(ga_data.device_model)
ga_data["group"] = enc_group.fit_transform(ga_data.group)

In [5]:
drop_list = ["isTrain"]
train = ga_data[ga_data.isTrain==1].drop( drop_list, axis=1)
true_classes = train.group
test = ga_data[ga_data.isTrain==0].drop( drop_list+["group"], axis=1)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(train
                                                   , true_classes, test_size=0.3)

In [7]:

train_drop = ["device_id","group"]

## We use the best single model from previous tests
XGB classifier 

In [14]:
# the parameters have been determined in the `predict_classe` notebook.
gbdt = GradientBoostingClassifier(loss='deviance'
                                  ,max_features=None
                                  , min_samples_leaf=800
                                  , learning_rate=0.005
                                  , n_estimators=700);

In [48]:
gbdt.fit(x_train, y_train)

GradientBoostingClassifier(init=None, learning_rate=0.005, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=800, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=700,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [52]:
probs_train = gbdt.predict_proba(x_train)
probs_test = gbdt.predict_proba(x_test)

In [54]:
print("Test MVA predictions on test and training set:\n")
print("Log loss on training set: ", log_loss(y_train,probs_train))
print("Log loss on test set: ", log_loss(y_test,probs_test))

Test MVA predictions on test and training set:

Log loss on training set:  2.39388310435
Log loss on test set:  2.40801068014


Performs largely identical.

## compare with linear model

In [196]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l1'
                        , C=0.6
                        , tol=0.000001
                        , solver= "liblinear"#"newton-cg"##'lbfgs'
                        , max_iter=2000
                        #, fit_intercept = False
                        #, class_weight = 'balanced'
                        #, multi_class='multinomial'
                       #,warm_start=True
                       )

In [9]:
#from sklearn.preprocessing import OneHotEncoder
grd_enc = OneHotEncoder()

In [197]:
lr.fit(grd_enc.fit_transform(x_train.drop(train_drop,axis=1)),y_train)

LogisticRegression(C=0.6, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=2000, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=1e-06,
          verbose=0, warm_start=False)

In [198]:
probs_lr_train = lr.predict_proba(grd_enc.transform(x_train.drop(train_drop,axis=1)))
probs_lr_test = lr.predict_proba(grd_enc.transform(x_test.drop(train_drop,axis=1)))

In [199]:
print("Test log-regression predictions on test and training set:\n")
print("Log loss on training set: ", log_loss(y_train,probs_lr_train))
print("Log loss on test set: ", log_loss(y_test,probs_lr_test))

Test log-regression predictions on test and training set:

Log loss on training set:  2.34649096859
Log loss on test set:  2.39829927098


## Try linear model ontop of gbdt

In [35]:
gbdt = joblib.load('trainedModels/final_gbdt_goNoEvts.pkl')

In [37]:
x_tr_lr, x_te_lr, y_tr_lr, y_te_lr = train_test_split(x_test, y_test, test_size=0.5)

In [38]:
grd_enc.fit(gbdt.apply(x_train)[:,:,0])
lr.fit(grd_enc.transform(gbdt.apply(x_tr_lr)[:,:,0]), y_tr_lr)

LogisticRegression(C=0.05, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=200, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [39]:
prob_gbdt_lr = lr.predict_proba(grd_enc.transform(gbdt.apply(x_te_lr)[:,:,0]))

In [40]:
prob_gbdt_lr_train = lr.predict_proba(grd_enc.transform(gbdt.apply(x_train)[:,:,0]))

In [41]:
print("Test log-regression predictions on test and training set:\n")
print("Log loss on training set: ", log_loss(y_train,prob_gbdt_lr_train))
print("Log loss on test set: ", log_loss(y_te_lr,prob_gbdt_lr))

Test log-regression predictions on test and training set:

Log loss on training set:  2.43667749883
Log loss on test set:  2.43049368416


# Save final xgb classifier

In [81]:
joblib.dump(gbdt, 'trainedModels/final_gbdt_goNoEvts.pkl', compress=3)

['trainedModels/final_gbdt_goNoEvts.pkl']

# Predict test sample and create submission file (part of it)

In [83]:
predictions = gbdt.predict_proba(test.drop(["device_id"], axis=1))

In [85]:
probas_gbdt_df = pd.DataFrame(predictions, index=test.device_id)
probas_gbdt_df.columns = enc_group.inverse_transform(probas_gbdt_df.columns)

In [88]:
probas_gbdt_df.to_csv("finalOutputs/gbdt_gaNoEvts_predictions.csv")

### can also use linear output


In [201]:
preds = lr.predict_proba(grd_enc.transform(test.drop("device_id",axis=1)))

In [202]:
probas_lr_df = pd.DataFrame(preds, index=test.device_id)
probas_lr_df.columns = enc_group.inverse_transform(probas_lr_df.columns)
probas_lr_df.to_csv("finalOutputs/lr_gaNoEvts_prediction.csv")